In [2]:
import os
import pandas as pd
import requests
import json
import psycopg2

def pg_select(sql):
    conn = psycopg2.connect(database='sdpdb', user='sdpuser', password='123456', host='200.31.175.88', port='5432')
    cur = conn.cursor()
    cur.execute(sql)
    rows = cur.fetchall()
    cols = [desc[0] for desc in cur.description]
    df = pd.DataFrame(rows,columns=cols)
    conn.commit()
    conn.close()
    return df

In [3]:
sql = """
select txn_dt,stlmnt_dt,bond_cd,buy_side_trdng_acnt_cd,sell_side_trdng_acnt_cd,yld_to_mrty,sum(nmnl_vol) as nmnl_vol,string_agg(dl_cd, ',') as dl_cds
from dpa.cbt_dl_dtl
where txn_dt = '2022-06-30' and chnl_tp!='10' and yld_to_mrty is not null and nmnl_vol is not null and TRDNG_MTHD_CD!='ESP'
group by txn_dt,stlmnt_dt,bond_cd,buy_side_trdng_acnt_cd,sell_side_trdng_acnt_cd,yld_to_mrty
"""
df_all =pg_select(sql)
df_all.to_csv("1.csv")

In [ ]:
df_all['prod'] = df_all.apply(lambda row:row['bond_cd']+str(row['stlmnt_dt'])+str(float(row['yld_to_mrty']))+str(float(row['nmnl_vol'])),axis=1)

In [50]:
import requests, json
import pandas as pd
import numpy as np

def convert(data):
    l = []
    d = {}
    for line in data:
        l.append(line[0])
        l.append(line[1])
    st = list(set(l))
    st.sort()
    for i in range(len(st)):
        d[st[i]] = i
    for i in range(len(data)):
        data[i][0] = d[data[i][0]]
        data[i][1] = d[data[i][1]]

    return d, data


# -------------------建立邻接表-----------------------------


def createGraph(data, n):
    g = [[] for i in range(n)]
    g1 = [[] for i in range(n)]  # 反向图
    for line in data:
        g[line[0]].append(line[1])
        g1[line[1]].append(line[0])
    for i in range(n):
        g[i].sort()  # 排序，为了输出有序
        g1[i].sort()
    return g, g1


# -------------------深度遍历找环--------------------------

# 6+1思路：利用反向图标记可能成为环的倒数第一个点，再进行深度为6的搜索，搜索到被标记的点则成环。


def dfs(g, k, p_o, visit, visit1, res, path):
    for i in range(len(g[k])):
        v = g[k][i]
        if v < p_o:  # 小于查询点p_o的点都不用访问
            continue

        if visit1[v] == -2 and visit[v] == 0:  # 当碰到倒数第一个点且这个点未被访问过，则找到一条路径
            path.append(v)  # 把倒数第一个点加入路径中
            length = len(path)
            if length >= 2:
                res[length - 2].append(path.copy())
            path.pop()

        if visit[v] == 1 or (visit1[v] != p_o and visit1[v] != -2):
            continue
        if len(path) == 4 or v == p_o:  # 当搜索长度到6 或者 访问到查询点p_o就不继续访问了
            continue
        visit[v] = 1  # 访问v
        path.append(v)
        dfs(g, v, p_o, visit, visit1, res, path)
        path.pop()
        visit[v] = 0  # 退出v点，取消访问标记


# ----------------------剪枝操作-----------------------------

# 3邻域剪枝思路：如果将图看作是无向图，一个点数为7的环中，距离起点最远的点距离不超过3


def dfs1(g, k, p_o, visit, visit1, length):
    for i in range(len(g[k])):
        if g[k][i] < p_o or visit[g[k][i]] == 1:  # 1.小于查询点p_o的点都不用访问了;  2.当前查询中已经访问过的点不用再访问;
            continue
        # p_o为当前查找环的点，把距离p_o小于等于3的点标记为p_o(也可以标记为1，不过结束查询p_o点之后要，重新标记为-1)
        visit1[g[k][i]] = p_o
        if length == 3:
            continue
        visit[g[k][i]] = 1
        dfs1(g, g[k][i], p_o, visit, visit1, length + 1)
        visit[g[k][i]] = 0






In [52]:
df_all

,txn_dt,stlmnt_dt,bond_cd,buy_side_trdng_acnt_cd,sell_side_trdng_acnt_cd,yld_to_mrty,nmnl_vol,dl_cds,prod
0,2022-06-30,2022-06-30,012103688,100036,401800,2.2016000000,20000000.0000000000,CBT20220630200786,0121036882022-06-30 00:00:002.201620000000.0
1,2022-06-30,2022-06-30,012103688,101929,401800,2.2016000000,10000000.0000000000,CBT20220630200162,0121036882022-06-30 00:00:002.201610000000.0
2,2022-06-30,2022-06-30,012103688,101929,413411,2.2016000000,10000000.0000000000,CBT20220630204563,0121036882022-06-30 00:00:002.201610000000.0
3,2022-06-30,2022-06-30,012103720,101169,403347,3.0792000000,20000000.0000000000,CBT20220630201109,0121037202022-06-30 00:00:003.079220000000.0
4,2022-06-30,2022-06-30,012103720,101815,102879,3.0792000000,30000000.0000000000,CBT20220630204144,0121037202022-06-30 00:00:003.079230000000.0
...,...,...,...,...,...,...,...,...,...
16714,2022-06-30,2022-07-08,200012,450402,100035,3.3620000000,2880000.0000000000,CBT20220630201066,2000122022-07-08 00:00:003.3622880000.0
16715,2022-06-30,2022-07-08,210005,450402,100035,3.3497000000,1670000.0000000000,CBT20220630206146,2100052022-07-08 00:00:003.34971670000.0
16716,2022-06-30,2022-07-08,210014,450402,102503,3.3277000000,1420000.0000000000,CBT20220630206022,2100142022-07-08 00:00:003.32771420000.0
16717,2022-06-30,2022-07-08,220008,450402,100035,3.2840000000,610000.0000000000,CBT20220630205600,2200082022-07-08 00:00:003.284610000.0


In [53]:
#prod区分相同债相同价格相同量
prod_lst = df_all['prod'].drop_duplicates().to_list()

grp_id = 0
df_res = pd.DataFrame()
for prod in prod_lst:
    df = df_all[df_all['prod']==prod]
    
    data = df[['buy_side_trdng_acnt_cd', 'sell_side_trdng_acnt_cd']].drop_duplicates().values.tolist()
    trans_dict, data = convert(data)
    df['buy_instn_cd'] = df['buy_side_trdng_acnt_cd'].map(trans_dict)
    df['sell_instn_cd'] = df['sell_side_trdng_acnt_cd'].map(trans_dict)
    n = len(trans_dict)
    g, g1 = createGraph(data, n)
    visit = [0 for i in range(n)]
    visit1 = [-1 for i in range(n)]  # 确定3邻域
    path = []
    res = [[] for i in range(4)]  # 长度分别为2,3，4，5，6，7的路径

    for i in range(n):
        # --------------3邻域剪枝----------------------------------
        # 分别遍历g,g1确定查询点i的3邻域
        # 把距离小于3的点的visit1标记为i，这样就确定了3邻域(不标记为1或者0是因为每次循环不用重新初始化visit1，为了省时间)
        dfs1(g, i, i, visit, visit1, 1)
        dfs1(g1, i, i, visit, visit1, 1)

        # --------------深度遍历找环--------------------------------
        for j in range(len(g1[i])):
            # 将倒数第一个点（6+1）visit1标记为-2，其实可以新建一个visit2来标记的，要容易理解些。
            visit1[g1[i][j]] = -2

        path.append(i)
        dfs(g, i, i, visit, visit1, res, path)  # 找查询点i的所有环
        path.pop()

        for j in range(len(g1[i])):
            visit1[g1[i][j]] = i  # 清除visit1 "-2" 的标记

            

    for r in res:
        for k in r:
            tmp = df[(df['buy_instn_cd'].isin(k)) & (df['sell_instn_cd'].isin(k))]
            if tmp['yld_to_mrty'].max() == tmp['yld_to_mrty'].min() and tmp['nmnl_vol'].max() == tmp['nmnl_vol'].min():
                tmp['grp_id'] = grp_id
                df_res = pd.concat([df_res, tmp])
                grp_id+=1


<ipython-input-53-8ea39f20727d>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['buy_instn_cd'] = df['buy_side_trdng_acnt_cd'].map(trans_dict)
<ipython-input-53-8ea39f20727d>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sell_instn_cd'] = df['sell_side_trdng_acnt_cd'].map(trans_dict)
<ipython-input-53-8ea39f20727d>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

In [54]:
df_res = df_res.reset_index(drop=True)

In [55]:
df_res

,txn_dt,stlmnt_dt,bond_cd,buy_side_trdng_acnt_cd,sell_side_trdng_acnt_cd,yld_to_mrty,nmnl_vol,dl_cds,prod,buy_instn_cd,sell_instn_cd,grp_id
0,2022-06-30,2022-06-30,150218,100013,103101,2.7350000000,50000000.0000000000,CBT20220630305070,1502182022-06-30 00:00:002.73550000000.0,0,2,0
1,2022-06-30,2022-06-30,150218,100036,100013,2.7350000000,50000000.0000000000,CBT20220630305045,1502182022-06-30 00:00:002.73550000000.0,1,0,0
2,2022-06-30,2022-06-30,150218,103101,100036,2.7350000000,50000000.0000000000,CBT20220630305044,1502182022-06-30 00:00:002.73550000000.0,2,1,0
3,2022-06-30,2022-06-30,180322,100013,101929,4.2018000000,260000000.0000000000,"CBT20220630302606,CBT20220630302621,CBT2022063...",1803222022-06-30 00:00:004.2018260000000.0,0,2,1
4,2022-06-30,2022-06-30,180322,100358,100013,4.2018000000,260000000.0000000000,"CBT20220630304634,CBT20220630304626,CBT2022063...",1803222022-06-30 00:00:004.2018260000000.0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
271,2022-06-30,2022-07-01,220210,101169,101987,3.0550000000,50000000.0000000000,CBT20220630300370,2202102022-07-01 00:00:003.05550000000.0,1,2,102
272,2022-06-30,2022-07-01,220210,101987,101169,3.0550000000,50000000.0000000000,CBT20220630300733,2202102022-07-01 00:00:003.05550000000.0,2,1,102
273,2022-06-30,2022-07-01,2205023,100013,100063,2.2600000000,10000000.0000000000,CBT20220630303819,22050232022-07-01 00:00:002.2610000000.0,0,1,103
274,2022-06-30,2022-07-01,2205023,100063,100662,2.2600000000,10000000.0000000000,CBT20220630304823,22050232022-07-01 00:00:002.2610000000.0,1,2,103


In [56]:
lst = []
for i in range(len(df_res)):
    tmp = df_res.loc[i,'dl_cds'].split(',')
    lst.extend(tmp)
lst = list(set(lst))
lst

['CBT20220630303814',
 'CBT20220630903209',
 'CBT20220630901028',
 'CBT20220630901456',
 'CBT20220630901148',
 'CBT20220630900736',
 'CBT20220630304284',
 'CBT20220630900580',
 'CBT20220630902288',
 'CBT20220630300803',
 'CBT20220630900673',
 'CBT20220630901620',
 'CBT20220630902691',
 'CBT20220630303816',
 'CBT20220630304398',
 'CBT20220630304829',
 'CBT20220630200501',
 'CBT20220630901027',
 'CBT20220630900808',
 'CBT20220630205325',
 'CBT20220630304774',
 'CBT20220630902649',
 'CBT20220630901341',
 'CBT20220630304810',
 'CBT20220630902680',
 'CBT20220630903769',
 'CBT20220630304634',
 'CBT20220630203790',
 'CBT20220630305044',
 'CBT20220630900735',
 'CBT20220630902430',
 'CBT20220630901453',
 'CBT20220630300991',
 'CBT20220630300733',
 'CBT20220630303171',
 'CBT20220630303819',
 'CBT20220630900774',
 'CBT20220630302726',
 'CBT20220630901166',
 'CBT20220630902399',
 'CBT20220630200496',
 'CBT20220630901870',
 'CBT20220630901151',
 'CBT20220630300269',
 'CBT20220630901651',
 'CBT20220

In [57]:
len(lst)

282

In [58]:
df_res.to_csv('对倒交易.csv')

In [11]:
data = df[['buy_side_trdng_acnt_cd', 'sell_side_trdng_acnt_cd']].drop_duplicates().values.tolist()
trans_dict, data = convert(data)
df['buy_instn_cd'] = df['buy_side_trdng_acnt_cd'].map(trans_dict)
df['sell_instn_cd'] = df['sell_side_trdng_acnt_cd'].map(trans_dict)
n = len(trans_dict)
g, g1 = createGraph(data, n)
visit = [0 for i in range(n)]
visit1 = [-1 for i in range(n)]  # 确定3邻域
path = []
res = [[] for i in range(4)]  # 长度分别为2,3，4，5，6，7的路径

for i in range(n):
    # --------------3邻域剪枝----------------------------------
    # 分别遍历g,g1确定查询点i的3邻域
    # 把距离小于3的点的visit1标记为i，这样就确定了3邻域(不标记为1或者0是因为每次循环不用重新初始化visit1，为了省时间)
    dfs1(g, i, i, visit, visit1, 1)
    dfs1(g1, i, i, visit, visit1, 1)

    # --------------深度遍历找环--------------------------------
    for j in range(len(g1[i])):
        # 将倒数第一个点（6+1）visit1标记为-2，其实可以新建一个visit2来标记的，要容易理解些。
        visit1[g1[i][j]] = -2

    path.append(i)
    dfs(g, i, i, visit, visit1, res, path)  # 找查询点i的所有环
    path.pop()

    for j in range(len(g1[i])):
        visit1[g1[i][j]] = i  # 清除visit1 "-2" 的标记





In [9]:
grp_id

0

In [15]:
res[3][0]

[0, 1, 11, 2, 21]

In [12]:
grp_id = 0
df_res = pd.DataFrame()
for r in res:
    for k in r:
        grp_id+=1

In [13]:
grp_id

13108063

In [12]:

for r in res:
    for k in r:
        tmp = df[(df['buy_instn_cd'].isin(k)) & (df['sell_instn_cd'].isin(k))]
        if tmp['yld_to_mrty'].max() == tmp['yld_to_mrty'].min() and tmp['nmnl_vol'].max() == tmp['nmnl_vol'].min():
            tmp['grp_id'] = grp_id
            df_res = pd.concat([df_res, tmp])
            grp_id+=1

<ipython-input-12-63f46c846f0f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['grp_id'] = grp_id


KeyboardInterrupt: 

In [13]:
df_res.sort_values(by='')

,txn_dt,buy_side_trdng_acnt_cd,sell_side_trdng_acnt_cd,yld_to_mrty,nmnl_vol,string_agg,buy_instn_cd,sell_instn_cd,grp_id
279,2022-06-30,100007,100477,3.0515000000,50000000.0000000000,CBT20220630900251,5,123,0
5600,2022-06-30,100477,100007,3.0515000000,50000000.0000000000,CBT20220630900179,123,5,0
558,2022-06-30,100012,102360,2.6250000000,100000000.0000000000,CBT20220630205150,10,304,1
9907,2022-06-30,102360,100012,2.6250000000,100000000.0000000000,CBT20220630205151,304,10,1
1091,2022-06-30,100013,416044,3.1575000000,10000000.0000000000,CBT20220630203186,11,1453,2
13104,2022-06-30,416044,100013,3.1575000000,10000000.0000000000,CBT20220630205661,1453,11,2
2354,2022-06-30,100036,411680,3.1425000000,40000000.0000000000,CBT20220630204000,21,1248,3
12716,2022-06-30,411680,100036,3.1425000000,40000000.0000000000,CBT20220630203984,1248,21,3
2701,2022-06-30,100047,101987,3.0500000000,50000000.0000000000,"CBT20220630902649,CBT20220630902647",22,243,4
8736,2022-06-30,101987,100047,3.0500000000,50000000.0000000000,CBT20220630305008,243,22,4
